In [1]:
import tensorflow as tf
from tensorflow.keras import regularizers
print(tf.__version__)

2.2.0


In [2]:
import tensorflow_datasets as tfds
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHESN6L/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHESN6L/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHESN6L/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np
train_data,test_data=imdb['train'],imdb['test']

training_sentences=[]
training_label=[]

testing_sentences=[]
testing_label=[]

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_label.append(l.numpy()) 

for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_label.append(l.numpy())

training_lables_final=np.array(training_label)
testing_labels_final=np.array(testing_label)


In [31]:
vocab_size=12000
embedding_dim=16
max_length=120
trunc_type='post'
oov_tok="<oov>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences=sequences,maxlen=max_length,truncating=trunc_type)
testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(sequences=testing_sequences,maxlen=max_length,truncating=trunc_type)
print(len(sequences))
print(len(testing_sequences))

25000
25000


In [0]:
training_target=[]
training_context=[]
for i in range(len(sequences)):
  for j in range(len(sequences[i])):
    training_target.append(sequences[i][j])
    if(j==0):
      training_context.append([sequences[i][j+1], sequences[i][j+2], sequences[i][j+3]])
    elif(j==len(sequences[i])-1):
      training_context.append([sequences[i][j-3], sequences[i][j-2], sequences[i][j-1]])
    elif(j==len(sequences[i])-2):
      training_context.append([sequences[i][j-2], sequences[i][j-1], sequences[i][j+1]])
    else:
      training_context.append([sequences[i][j-1], sequences[i][j+1], sequences[i][j+2]])


In [0]:
testing_target=[]
testing_context=[]
for i in range(len(testing_sequences)):
  for j in range(len(testing_sequences[i])):
    testing_target.append(testing_sequences[i][j])
    if(j==0):
      testing_context.append([testing_sequences[i][j+1], testing_sequences[i][j+2], testing_sequences[i][j+3]])
    elif(j==len(testing_sequences[i])-1):
      testing_context.append([testing_sequences[i][j-3], testing_sequences[i][j-2], testing_sequences[i][j-1]])
    elif(j==len(testing_sequences[i])-2):
      testing_context.append([testing_sequences[i][j-2], testing_sequences[i][j-1], testing_sequences[i][j+1]])
    else:
      testing_context.append([testing_sequences[i][j-1], testing_sequences[i][j+1], testing_sequences[i][j+2]])
training_target_final=np.array(training_target)
training_context_final=np.array(training_context)
testing_target_final=np.array(testing_target)
testing_context_final=np.array(testing_context)

In [34]:
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])
def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])
print(decode_review(training_sentences[3]))
print(training_sentences[3])
print(len(word_index))
print(word_index["large"])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
b'This is the kind of film for a snowy Sunday afternoon when the r

In [35]:

from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.layers import Embedding,Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

embedding_dim=16
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=3))
model.add(Flatten())
model.add(Dense(6,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# view model summary
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 3, 16)             192000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 48)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 294       
_________________________________________________________________
dense_6 (Dense)              (None, 12000)             84000     
Total params: 276,294
Trainable params: 276,294
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
num_epochs=15
history=model.fit(training_context_final,training_target_final,batch_size=25000,epochs=num_epochs,validation_data=(testing_context_final,testing_target_final))

Epoch 1/15
242/242 [==============================] - 64s 264ms/step - loss: 7.7735 - accuracy: 0.0342 - val_loss: 6.4680 - val_accuracy: 0.0558
Epoch 2/15
242/242 [==============================] - 64s 265ms/step - loss: 6.4797 - accuracy: 0.0554 - val_loss: 6.4068 - val_accuracy: 0.0558
Epoch 3/15
242/242 [==============================] - 65s 267ms/step - loss: 6.4258 - accuracy: 0.0555 - val_loss: 6.3563 - val_accuracy: 0.0572
Epoch 4/15
242/242 [==============================] - 65s 267ms/step - loss: 6.3525 - accuracy: 0.0679 - val_loss: 6.2384 - val_accuracy: 0.0852
Epoch 5/15
242/242 [==============================] - 65s 267ms/step - loss: 6.1617 - accuracy: 0.0940 - val_loss: 5.9927 - val_accuracy: 0.1102
Epoch 6/15
242/242 [==============================] - 65s 268ms/step - loss: 5.9135 - accuracy: 0.1198 - val_loss: 5.7654 - val_accuracy: 0.1344
Epoch 7/15
242/242 [==============================] - 65s 268ms/step - loss: 5.7264 - accuracy: 0.1384 - val_loss: 5.6175 - val_ac

In [37]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape)
print(weights[0])

(12000, 16)
[-0.02034237  0.02659469  0.00271029 -0.00903394 -0.04101688 -0.02434638
  0.03959414 -0.00675958 -0.01314499  0.01992316 -0.02450718  0.0412205
 -0.01057801 -0.04735582  0.01818277 -0.02035305]


In [39]:
def word_to_vec_map(word):
  return (weights[word_index[word]])
print(word_to_vec_map("man"))
print(word_to_vec_map("woman"))

[ 0.1303756   0.35489193 -0.6018018  -0.1583862  -0.62503725 -0.31603724
  0.39467123  0.12872306  0.78713113  0.15401688 -0.00230335  0.37285587
  0.18536443  0.27989727  0.31603816  0.2652128 ]
[ 0.09517043  0.18084747 -0.56795335 -0.05641099 -0.6964467  -0.3820631
  0.43506452  0.26050898  0.82597584  0.24597847  0.02744375  0.36315998
  0.3007749   0.3103087   0.31499344  0.26753712]


In [0]:
def cosine_similarity(u, v):
    
    distance = 0.0

    dot = np.dot(u, v)

    norm_u = np.sqrt(np.sum(u * u))
    
    norm_v = np.sqrt(np.sum(v * v))
    
    cosine_similarity = dot / (norm_u * norm_v)
    
    return cosine_similarity

In [47]:
king=weights[word_index["king"]]
queen=weights[word_index["queen"]]
print(cosine_similarity(king,queen))

0.94621897


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

Now using these trained embeddings to classify reviews

In [0]:
num_words=vocab_size
embedding_matrix=np.zeros((num_words,embedding_dim))
for word,i in word_index.items():
  if(i>=num_words):
    continue
  embedding_vector=word_to_vec_map(word)
  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector


In [44]:
from tensorflow.keras.layers import Dense,Flatten,Dropout,GRU
from tensorflow.keras.layers import Embedding,Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import Constant

model = Sequential()
embedding_layer=Embedding(vocab_size,embedding_dim,embeddings_initializer=Constant(embedding_matrix),input_length=max_length,trainable=False)
model.add(embedding_layer)
model.add(GRU(64))
model.add(Dropout(0.2))
#model.add(GRU(32))
#model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# view model summary
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 120, 16)           192000    
_________________________________________________________________
gru_8 (GRU)                  (None, 64)                15744     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 207,809
Trainable params: 15,809
Non-trainable params: 192,000
_________________________________________________________________
None


In [45]:
num_epochs=40
model.fit(padded,training_lables_final,batch_size=128,epochs=num_epochs,validation_data=(testing_padded,testing_labels_final),verbose=2)

Epoch 1/40
196/196 - 3s - loss: 0.6926 - accuracy: 0.5189 - val_loss: 0.6900 - val_accuracy: 0.5336
Epoch 2/40
196/196 - 2s - loss: 0.6814 - accuracy: 0.5649 - val_loss: 0.6740 - val_accuracy: 0.5856
Epoch 3/40
196/196 - 2s - loss: 0.6660 - accuracy: 0.5979 - val_loss: 0.6674 - val_accuracy: 0.6009
Epoch 4/40
196/196 - 2s - loss: 0.6498 - accuracy: 0.6230 - val_loss: 0.6629 - val_accuracy: 0.5984
Epoch 5/40
196/196 - 2s - loss: 0.6415 - accuracy: 0.6322 - val_loss: 0.6484 - val_accuracy: 0.6247
Epoch 6/40
196/196 - 2s - loss: 0.6285 - accuracy: 0.6522 - val_loss: 0.6329 - val_accuracy: 0.6417
Epoch 7/40
196/196 - 2s - loss: 0.6173 - accuracy: 0.6589 - val_loss: 0.6216 - val_accuracy: 0.6570
Epoch 8/40
196/196 - 2s - loss: 0.6116 - accuracy: 0.6649 - val_loss: 0.6214 - val_accuracy: 0.6606
Epoch 9/40
196/196 - 2s - loss: 0.6021 - accuracy: 0.6740 - val_loss: 0.6234 - val_accuracy: 0.6508
Epoch 10/40
196/196 - 2s - loss: 0.5940 - accuracy: 0.6827 - val_loss: 0.5980 - val_accuracy: 0.6776